In [3]:
import math
from typing import List

import torch
import torch.nn.functional as F

from torch import nn
# NF4라는 저정밀도 포맷을 사용해 선형 계층의 가중치를 양자화하는 모듈들을 가져옴
from torchao.dtypes.nf4tensor import linear_nf4, to_nf4
# LoRA 계층에서 NF4 양자화 연산을 처리하는 모듈 가져오기 (하지만, 여기선 실제로 사용되진 않음)
from torchtune.modules.low_precision import _register_nf4_dispatch_ops
# LoRA와 관련된 어댑터 모듈을 가져옴 (LoRA의 특성에 맞게 작동하도록 하는 모듈)
from torchtune.modules.peft import AdapterModule

In [ ]:
class LoRALinear(nn.modules, AdapterModule):
    '''
    LoRA (Low-Rank Adaptation) 선형 계층을 정의하는 클래스.
    LoRA는 대규모 언어 모델의 적응을 위한 기법으로, 저차원 행렬 분해를 통해 적은 파라미터로 학습을 가능하게 한다.
    기존의 선형 변환 x -> Wx에 대해 LoRA는 W0x + (alpha / r)BAx로 변경하여 학습할 수 있다.

    Args:
        in_dim (int): 입력 차원 수
        out_dim (int): 출력 차원 수
        rank (int): 저차원 행렬 분해에서의 rank
        alpha (float): 저차원 행렬 분해의 스케일링 계수
        dropout (float): 드롭아웃 확률 (기본값: 0.0)
        use_bias (bool): 원래 선형 계층에 편향(bias)를 포함할지 여부 (기본값: False)
        quantize_base (bool): 기본 선형 가중치를 양자화할지 여부 (기본값: False) 
    '''

    def __init__(
            self,
            in_dim: int,
            out_dim: int,
            rank: int,
            alpha: float,
            dropout: float = 0.0,
            use_bias: bool = False,
            quantize_base: bool = False,
    ):
        super().__init__() # 부모 클래스(nn.module, AdapterModule) 초기화
        self.in_dim = in_dim
        self.rank = rank
        self.alpha = alpha
        self.out_dim = out_dim
        self.use_bias = use_bias
        self._quantize_base = quantize_base

        # 선형 계층의 가중치와 편향을 생성
        weight, bias = self.create_weight_and_bias()

        # self.disabled는 LoRA 어뎁터를 끌 수 있는 플래그로 모델을 비교하기 위해 끌 때 사용
        self.disabled = False

        # 가중치와 편향은 nn.Parameter로 등록 (학습 가능한 파라미터로 지정)
        self.register_parameter("weight", nn.Parameter(weight))
        self.register_parameter(
            "bias", nn.Parameter(bias) if bias is not None else None
        )

        # dropout Layer 추가(0일 경우 Identity로 처리)
        self.dropout = nn.Dropout(p=dropout) if dropout > 0.0 else nn.Identity()

        # LoRA A와 B 행렬을 Linear Layer로 정의
        self.lora_a = nn.Linear(in_features=in_dim, out_features=rank, bias=False)
        self.lora_b = nn.Linear(in_features=rank, out_features=out_dim, bias=False)

        # 가중치가 병합되었는지 여부
        self.merged = False

        self.initialize_parameters()
    
    def initialize_parameters(self):
        # LoRA A와 B 행렬을 초기화하는 함소 호출
        _lora_a_init_params(self.lora_a)
        _lora_b_init_params(self.lora_b)
    
    def _create_weight_and_bias(self):
        '''
        Linear Layer의 가중치와 편향을 생성하는 함수. 양자화를 사용할 경우 NF4 데이터 타입을 사용
        '''
        in_dim, out_dim, use_bias = self.in_dim, self.out_dim, self.use_bias

        # nn.Linear로부터 가중치와 편향을 생성
        linear = nn.Linear(in_features=in_dim, out_features=out_dim, bias=use_bias)

        # 양자화 설정에 따라 가중치를 NF4로 변환하거나 그대로 사용
        weight = linear.weight if not self._quantize_base else to_nf4(linear.weight)

        #편향 처리
        bias = None
        if self.use_bias:
            if self._quantize_base:
                raise NotImplementedError(
                    "양자화된 LoRALinear는 bias를 지원하지 않습니다"
                )
            bias = linear.bias
        
        return weight, bias
    
    def adapter_prams(self) -> List[str]:
        '''
        LoRA에서 학습해야 하는 저차원 행렬의 가중치(lora_A와 lora_B의 weight)를 명시적으로 지정해서, 
        모델의 다른 파라미터들과 구분하고, 학습 가능한 파라미터로 관리할 수 있도록 어댑터 파라미터로 반환하는 역할을 합니다. 
        이를 통해, 모델 전체가 아니라 오직 LoRA 부분만 학습되도록 제어할 수 있음
        그리고 가중치 이름을 텍스트 형태로 반환합니다. 
        이를 리스트로 반환하는 이유는, 이 리스트가 학습 가능한 파라미터들의 이름을 모아두는 일종의 레퍼런스 역할
        '''

        return ["lora_a.weight", "lora_b.weight"]
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        주어진 입력 x에 대해 LoRA 변환을 적용하는 함수.

        Args:
            x (torch.Tensor): 입력 텐서, 크기는 (..., in_dim)
        
        Returns:
            torch.Tensor: 출력 텐서, 크기는 (..., out_dim)
        """
        # 양자화된 가중치 사용할 경우 NF4 연산 적용
        if self._quantize_base:
            out = linear_nf4(input=x, weight=self.weight)
        else:
            # 일반 선형 변환 (기존 가중치로 계산)
            out = F.linear(x, self.weight, self.bias)
        
        # LoRA 어댑터가 비활성화된 경우, 기존 선형 변환만 반환
        if self.disabled:
            return out

        # LoRA A 행렬에 드롭아웃을 적용한 후 저차원 변환
        lora_out = self.lora_a(self.dropout(x))
        
        # LoRA B 행렬을 통해 저차원에서 다시 출력 차원으로 확장
        lora_out = (self.alpha / self.rank) * self.lora_b(lora_out)

        # 기존 선형 출력과 LoRA 변환을 더해 최종 결과 반환
        return out + lora_out           

def _lora_a_init_params(x: nn.Linear) -> None:
    '''
    LoRA A 행렬을 Kaiming Uniform 방식으로 초기화.
    '''
    nn.init.kaiming_uniform_(x.weight, a=math.sqrt(5))


def _lora_b_init_params(x: nn.Linear) -> None:
    """
    LoRA B 행렬을 0으로 초기화.
    """
    nn.init.zeros_(x.weight)

**LoRA**의 저차원 행렬을 이용한 변환 과정과, `lora_a` 및 `lora_b`가 어떻게 작동하는지 하나씩 분석하면 아래와 같음

먼저, **`lora_a`**와 **`lora_b`**는 각각 **저차원 변환**과 **출력 확장**을 담당하는 선형 계층(`nn.Linear`)입니다. LoRA는 **저차원 행렬 분해**를 통해 모델의 파라미터 수를 줄이면서도 성능을 유지하는 기법인데, 이를 위해 두 개의 선형 변환(`lora_a`와 `lora_b`)을 사용합니다.

### 1. **`lora_a(self.dropout(x))` 설명**

```python
lora_out = self.lora_a(self.dropout(x))
```

- **`lora_a`**: `lora_a`는 `nn.Linear(in_features=in_dim, out_features=rank)`로 정의되어 있습니다. 즉, 입력 차원의 데이터를 **저차원(rank)**으로 변환하는 선형 계층입니다. 
- **`dropout(x)`**: 입력 데이터 `x`에 드롭아웃을 적용합니다. `x`는 보통 `(batch_size, in_features)` 크기를 가지며, 이 `x`에 드롭아웃을 적용해 **일부 요소를 확률적으로 0으로 만든 후**, 이를 `lora_a`에 전달합니다.
  - 드롭아웃을 사용하는 이유는 **학습 시 과적합을 방지**하고, **모델의 일반화 성능을 향상**시키기 위해서입니다.
  - 드롭아웃이 적용된 입력 데이터는 **저차원 변환을 위한 입력**으로 사용됩니다.
  
- **`lora_a(self.dropout(x))`**: 드롭아웃을 적용한 입력 데이터를 `lora_a`로 전달하면, `lora_a`는 이를 저차원 공간으로 변환합니다. 이때 `lora_a`는 **입력 차원(in_dim)**을 **저차원(rank)**으로 변환하는 역할을 합니다.  
  - 예를 들어, `x`의 크기가 `(batch_size, in_features)`라면, `lora_a`는 **입력 차원(in_features)을 `rank` 크기의 저차원으로 변환**하여 결과로 `(batch_size, rank)` 크기의 텐서를 출력합니다.

### 2. **`lora_b(lora_out)` 설명**

```python
lora_out = (self.alpha / self.rank) * self.lora_b(lora_out)
```

- **`lora_b`**: `lora_b`는 `nn.Linear(in_features=rank, out_features=out_dim)`로 정의되어 있습니다. 즉, 저차원(rank)에서 다시 **출력 차원(out_dim)**으로 확장하는 선형 계층입니다. 
- **`lora_b(lora_out)`**: `lora_out`는 **`lora_a`를 통해 얻어진 저차원 표현**입니다. 이 저차원 표현을 다시 **출력 차원으로 확장**하는 것이 `lora_b`의 역할입니다. `lora_b`는 **저차원(rank)**에서 **출력 차원(out_dim)**으로 변환하는 선형 변환입니다.
  - 예를 들어, `lora_a`가 출력한 `(batch_size, rank)` 텐서를 `lora_b`는 **출력 차원(out_dim)**으로 확장하여 `(batch_size, out_dim)` 크기의 텐서로 변환합니다.

- **스케일링 적용**: 변환된 결과에 **스케일링**이 적용됩니다.
  ```python
  (self.alpha / self.rank)
  ```
  - `self.alpha`: LoRA에서 저차원 변환의 영향을 조절하는 **스케일링 계수**입니다. 저차원 변환의 크기를 조정하여 **학습이 불안정해지지 않도록** 합니다.
  - `self.rank`: 저차원 행렬 분해에서의 **저차원 차원(rank)**입니다. `alpha / rank`로 나누는 것은 저차원 변환의 결과를 **적절히 축소**하여 모델이 기존 출력에 더할 때 **너무 큰 영향을 미치지 않도록** 조정하는 역할을 합니다.

### 3. **숨겨진 과정**

#### 1. **`lora_a`와 `lora_b` 내부 연산**
- **`lora_a`**는 **선형 변환**입니다. `lora_a`는 주어진 입력 텐서 `x`와 **가중치 행렬(weight matrix)**을 곱한 후 출력으로 내보냅니다. 이는 일반적인 **선형 변환**(`Wx + b`에서 편향 `b`는 여기서 사용하지 않음)입니다. 
  - 이때 `lora_a`의 가중치 행렬은 크기가 `(rank, in_dim)`입니다. 즉, **입력 차원(in_dim)을 저차원(rank)**으로 변환하는 가중치 행렬입니다.
  
- **`lora_b`**도 마찬가지로 **선형 변환**을 수행합니다. `lora_b`의 가중치 행렬은 크기가 `(out_dim, rank)`입니다. 즉, **저차원(rank)을 다시 출력 차원(out_dim)**으로 확장하는 가중치 행렬입니다.

#### 2. **선형 변환의 수학적 표현**
`lora_a`와 `lora_b`는 다음과 같이 작동합니다:
- **`lora_a`**: 
  \[
  lora\_out = lora\_a(x) = xW_a^T
  \]
  여기서 `x`는 `(batch_size, in_dim)` 크기의 입력 텐서, `W_a`는 `(rank, in_dim)` 크기의 가중치 행렬입니다. 결과로 `lora_out`은 `(batch_size, rank)` 크기를 가집니다.
  
- **`lora_b`**:
  \[
  lora\_out = lora\_b(lora\_out) = lora\_outW_b^T
  \]
  여기서 `lora_out`은 `(batch_size, rank)` 크기의 저차원 텐서, `W_b`는 `(out_dim, rank)` 크기의 가중치 행렬입니다. 이 선형 변환의 결과로 **출력 차원으로 확장된 `(batch_size, out_dim)` 크기의 텐서**가 생성됩니다.

#### 3. **최종 스케일링 적용**
- 변환된 결과에 대해 `(self.alpha / self.rank)` 스케일링을 적용하여 **LoRA의 영향**을 적절히 조정합니다.

---

### 요약:
1. **`lora_a(self.dropout(x))`**: 입력 데이터 `x`에 드롭아웃을 적용한 후, 이를 **저차원(rank)**으로 변환하는 선형 변환(`lora_a`)을 수행합니다. 이때 `lora_a`는 **입력 차원에서 저차원으로 변환**하는 역할을 합니다.
  
2. **`lora_b(lora_out)`**: `lora_a`의 출력인 **저차원 표현**을 다시 **출력 차원(out_dim)**으로 확장하는 선형 변환(`lora_b`)을 수행합니다. `lora_b`는 **저차원(rank)에서 출력 차원으로 변환**하는 역할을 합니다.

3. **최종적으로** `lora_b`의 출력에 스케일링을 적용하여, LoRA 변환의 영향이 적절하게 조정된 상태로 **기존 선형 변환 결과에 더해집니다**.